In [1]:
import re
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('Data/Spam Data.csv')

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df.iloc[2,1]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [4]:
wordnet=WordNetLemmatizer()
corpus = []
for i in range(len(df['Message'])):
    review = re.sub('[^a-zA-Z]', ' ', df['Message'][i]).lower().split()
    review = [wordnet.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    corpus.append(review)

In [5]:
corpus = [" ".join(review) for review in corpus]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=3000)
X = tf.fit_transform(corpus)
y = pd.get_dummies(df['Category'])
y = y.iloc[:,1].values

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=0)

In [8]:
svc = SVC(kernel='rbf')
svc.fit(x_train, y_train)
y_pred = svc.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[955   0]
 [ 19 141]]
0.9829596412556054


In [9]:
print(classification_report(y_test, svc.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       955
           1       1.00      0.88      0.94       160

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
#Saving SVM Model
file_name = "Spam_Classifier.sav"
pickle.dump(svc,open(file_name,'wb'))

In [ ]:
#Saving vectorizer
file_name = "tfidf.pickle"
pickle.dump(tf,open(file_name,'wb'))

In [2]:
vectors = pickle.load(open('tfidf.pickle', 'rb'))

In [ ]:
models = pickle.load(open('SVC_Model.pickle', 'rb'))